In [1]:
import pandas as pd
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from collections import defaultdict
import tqdm
import random

In [2]:
transactions_path = f"transactions.csv"

In [3]:
transactions_data = pd.read_csv(transactions_path)

In [42]:
number_of_lines = 32109415
sample_num = 1109415
def skip_line(line):
    if line == 0:
        return False
    return random.random() > (sample_num / number_of_lines)

In [43]:
transactions_sampled = pd.read_csv(transactions_path, skiprows=skip_line)

# Label generation

In [44]:
positive_data = transactions_sampled[['client_id', 'material', 'chq_date']]

In [45]:
client_date_sales = defaultdict(lambda: defaultdict(set))

for _, row in tqdm.tqdm(positive_data.iterrows()):
    client_date_sales[row.client_id][row.chq_date].add(row.material)

1110580it [02:12, 8369.74it/s]


In [46]:
def check_date(client, material, date):
    return material in client_date_sales[client][date]

In [48]:
clients = np.array(transactions_sampled.client_id.unique())
materials = np.array(transactions_sampled.material.unique())
dates = np.array(transactions_sampled.chq_date.unique())

In [49]:
negative_data = []

for client in tqdm.tqdm(clients):
    dates_sample = np.random.choice(dates, 4, replace=False)
    for date in dates_sample:
        materials_sample = np.random.choice(materials, 3, replace=False)
        for material in materials_sample:
            if not check_date(client, material, date):
                negative_data.append((client, material, date))
    

100%|███████████████████████████████████████████████████████████████████████████| 92272/92272 [08:02<00:00, 191.04it/s]


In [54]:
negative_frame = pd.DataFrame(negative_data, columns=['client_id', 'material', 'chq_date'])

In [56]:
negative_frame.to_csv('negative.csv')
positive_data.to_csv('positive.csv')

# Merging

In [58]:
materials_enc = pd.read_csv('materials_encoded.csv').drop(columns=['Unnamed: 0'])

In [60]:
client_frame = pd.read_csv('clients.csv')

In [63]:
client_frame['gender'] = client_frame['gender'].apply(lambda x: x=='M')

In [65]:
client_frame['city'] = client_frame['city'].apply(lambda x: 0 if x == 'Moscow' else (1 if x =='St. Petersburg' else 2))

In [66]:
client_frame

,client_id,gender,city,birthyear
0,1a47d62dddacc03fe90c15652f7ae1a4,True,2,1990.0
1,9cb909f701d25d548e953bff81192b56,False,2,1969.0
2,d5da0f5b841b4f83383202807995027a,True,2,1976.0
3,13ed7f16810b17b8cee6de834ac79a48,False,0,1966.0
4,215fe3ea7d5bf0415e5504e2a7f33551,False,2,1988.0
...,...,...,...,...
99990,9e7e66fc2c2dcfd8977bc0cffeff8eb5,True,2,1985.0
99991,e261c97d40ab839926c712b4aa6cefd8,False,1,1982.0
99992,678ef46f41f8c89f7e8578c9e9592e3e,True,2,1964.0
99993,274facf0609ceba803f5853bd2222208,False,2,1951.0


In [85]:
merged_positive = positive_data.merge(materials_enc, on='material').merge(client_frame, on='client_id')

In [87]:
merged_negative = negative_frame.merge(materials_enc, on='material').merge(client_frame, on='client_id')

In [84]:
def fix_date(frame):
    dates = pd.to_datetime(frame['chq_date'])  - pd.Timestamp('2016-10-04 00:00:00')
    frame['chq_date'] = dates.dt.days
    return frame

In [89]:
merged_positive = fix_date(merged_positive)
merged_negative = fix_date(merged_negative)

In [93]:
merged_positive['label'] = 1
merged_negative['label'] = 0

In [94]:
merged = pd.concat([merged_positive, merged_negative])

In [95]:
merged

,client_id,material,chq_date,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco,gender,city,birthyear,label
0,4d68fdddb5deaa496ce485bd51929ab1,d66c44ca2800163901f22ec5fb4f1021,28,0,10,241,871,2891,0,0,True,1,1966.0,1
1,4d68fdddb5deaa496ce485bd51929ab1,6278f48a43fb0a23036b3a11c92e726c,316,0,27,376,1171,1386,0,0,True,1,1966.0,1
2,4d68fdddb5deaa496ce485bd51929ab1,5e3039a269ff005f2c76cea16c0cf783,102,0,35,189,569,2730,0,0,True,1,1966.0,1
3,4d68fdddb5deaa496ce485bd51929ab1,0b689fd70e79d5010e397b4c453e8460,61,0,32,360,1129,3825,0,0,True,1,1966.0,1
4,4d68fdddb5deaa496ce485bd51929ab1,763e061fd79190b901082035761a4a83,305,0,35,119,430,3856,0,0,True,1,1966.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106847,ee6b048e2aa02d49bb866e89dac444c9,9fff1f475ee1364ddaafdd2991be76dd,102,0,5,26,390,3110,0,0,False,2,2003.0,0
1106848,ee6b048e2aa02d49bb866e89dac444c9,8bbabf6269bb646e62e34ffbf1c2c4de,278,0,10,347,1371,1309,0,0,False,2,2003.0,0
1106849,ee6b048e2aa02d49bb866e89dac444c9,2338ab0f346c8050d78b955c653d001e,278,0,15,228,1453,1686,0,0,False,2,2003.0,0
1106850,ee6b048e2aa02d49bb866e89dac444c9,fcf72b8b2978f8d931761f77b3071b04,143,1,52,134,792,2396,0,0,False,2,2003.0,0


In [92]:
from sklearn.preprocessing import LabelEncoder

In [96]:
clients_enc = LabelEncoder()
clients_enc.fit(merged.client_id)
merged['client_id'] = clients_enc.transform(merged.client_id)

In [99]:
mat_enc = LabelEncoder()
mat_enc.fit(merged.material)
merged['material'] = mat_enc.transform(merged.material)

In [101]:
merged.to_csv('merged.csv')

# Train-test split

In [111]:
train_frame = merged[merged.chq_date < 250]
eval_frame = merged[(merged.chq_date >= 250) & (merged.chq_date < 300)]
test_frame = merged[merged.chq_date >= 300]

In [126]:
y_train = train_frame['label'].to_numpy()
X_train = train_frame.drop(columns='label').to_numpy()
y_eval = eval_frame['label'].to_numpy()
X_eval = eval_frame.drop(columns='label').to_numpy()
y_test = test_frame['label'].to_numpy()
X_test = test_frame.drop(columns='label').to_numpy()

In [127]:
import xgboost

In [142]:
boost = xgboost.XGBClassifier(n_estimators=300, max_depth=3, learning_rate=0.1)

In [143]:
boost.fit(X_train, y_train, eval_set=((X_eval, y_eval),), eval_metric='auc')

[0]	validation_0-auc:0.65702
[1]	validation_0-auc:0.68666
[2]	validation_0-auc:0.68666
[3]	validation_0-auc:0.69225
[4]	validation_0-auc:0.69586
[5]	validation_0-auc:0.69943
[6]	validation_0-auc:0.70591
[7]	validation_0-auc:0.70398
[8]	validation_0-auc:0.70792
[9]	validation_0-auc:0.71083
[10]	validation_0-auc:0.71392
[11]	validation_0-auc:0.71745
[12]	validation_0-auc:0.72119
[13]	validation_0-auc:0.72073
[14]	validation_0-auc:0.72273
[15]	validation_0-auc:0.72359
[16]	validation_0-auc:0.72316
[17]	validation_0-auc:0.72573
[18]	validation_0-auc:0.72694
[19]	validation_0-auc:0.72927
[20]	validation_0-auc:0.73011
[21]	validation_0-auc:0.73067
[22]	validation_0-auc:0.73228
[23]	validation_0-auc:0.73285
[24]	validation_0-auc:0.73872
[25]	validation_0-auc:0.73962
[26]	validation_0-auc:0.74190
[27]	validation_0-auc:0.74372
[28]	validation_0-auc:0.74401
[29]	validation_0-auc:0.74436
[30]	validation_0-auc:0.74473
[31]	validation_0-auc:0.74562
[32]	validation_0-auc:0.74614
[33]	validation_0-au

[268]	validation_0-auc:0.79763
[269]	validation_0-auc:0.79769
[270]	validation_0-auc:0.79771
[271]	validation_0-auc:0.79776
[272]	validation_0-auc:0.79784
[273]	validation_0-auc:0.79790
[274]	validation_0-auc:0.79790
[275]	validation_0-auc:0.79792
[276]	validation_0-auc:0.79809
[277]	validation_0-auc:0.79812
[278]	validation_0-auc:0.79814
[279]	validation_0-auc:0.79817
[280]	validation_0-auc:0.79825
[281]	validation_0-auc:0.79825
[282]	validation_0-auc:0.79830
[283]	validation_0-auc:0.79842
[284]	validation_0-auc:0.79842
[285]	validation_0-auc:0.79857
[286]	validation_0-auc:0.79864
[287]	validation_0-auc:0.79878
[288]	validation_0-auc:0.79893
[289]	validation_0-auc:0.79893
[290]	validation_0-auc:0.79893
[291]	validation_0-auc:0.79894
[292]	validation_0-auc:0.79899
[293]	validation_0-auc:0.79905
[294]	validation_0-auc:0.79923
[295]	validation_0-auc:0.79931
[296]	validation_0-auc:0.79935
[297]	validation_0-auc:0.79946
[298]	validation_0-auc:0.79948
[299]	validation_0-auc:0.79957


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [144]:
y_pred = boost.predict(X_test)

In [147]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.7126984780537245

# Hyperparameter tuning

In [154]:
# learning_rates = 10.0**np.random.uniform(-3, 0, size=10)
n_trees = [50, 100, 300, 500]
max_depths = [3, 5, 7, 9]

In [155]:
best_params = {}
best_score = 0.0
for n_tree in n_trees:
    for max_depth in max_depths:
        params = {'n_estimators': n_tree, 'max_depth': max_depth}
        boost = xgboost.XGBClassifier(learning_rate=0.1, **params)
        boost.fit(X_train, y_train)
        y_pred = boost.predict(X_eval)
        score = roc_auc_score(y_eval, y_pred)
        if score > best_score:
            best_score = score
            best_params = params
        print(f'Params: {params}, score: {score}')

Params: {'n_estimators': 50, 'max_depth': 3}, score: 0.6811082680981527
Params: {'n_estimators': 50, 'max_depth': 5}, score: 0.7091009580313319
Params: {'n_estimators': 50, 'max_depth': 7}, score: 0.7276401008914825
Params: {'n_estimators': 50, 'max_depth': 9}, score: 0.7483135615194264
Params: {'n_estimators': 100, 'max_depth': 3}, score: 0.6962306015082145
Params: {'n_estimators': 100, 'max_depth': 5}, score: 0.7241781825475228
Params: {'n_estimators': 100, 'max_depth': 7}, score: 0.7460870228901798
Params: {'n_estimators': 100, 'max_depth': 9}, score: 0.758300371257458
Params: {'n_estimators': 300, 'max_depth': 3}, score: 0.7195749849571162
Params: {'n_estimators': 300, 'max_depth': 5}, score: 0.7432415717365073
Params: {'n_estimators': 300, 'max_depth': 7}, score: 0.7611012856412722
Params: {'n_estimators': 300, 'max_depth': 9}, score: 0.7702148019041779
Params: {'n_estimators': 500, 'max_depth': 3}, score: 0.7265411266867305
Params: {'n_estimators': 500, 'max_depth': 5}, score: 0.

# Money Estimation

In [167]:
y_pred = boost.predict(X_test)

In [168]:
reversed_test = test_frame.copy()

In [169]:
reversed_test['client_id'] = clients_enc.inverse_transform(reversed_test['client_id'])

In [170]:
reversed_test['material'] = mat_enc.inverse_transform(reversed_test['material'])

In [171]:
reversed_test['importance'] = y_pred

In [172]:
reversed_test

,client_id,material,chq_date,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco,gender,city,birthyear,label,importance
1,4d68fdddb5deaa496ce485bd51929ab1,6278f48a43fb0a23036b3a11c92e726c,316,0,27,376,1171,1386,0,0,True,1,1966.0,1,1
4,4d68fdddb5deaa496ce485bd51929ab1,763e061fd79190b901082035761a4a83,305,0,35,119,430,3856,0,0,True,1,1966.0,1,1
7,4d68fdddb5deaa496ce485bd51929ab1,0d170a25e8f8c5ae206e4b801ffcd3d2,354,0,46,122,501,1101,0,0,True,1,1966.0,1,1
9,4d68fdddb5deaa496ce485bd51929ab1,61780dd011a4357cbb38d5b9433f8cf4,354,0,8,333,739,1968,0,0,True,1,1966.0,1,1
17,4d68fdddb5deaa496ce485bd51929ab1,995cadd7922027c0df05f0bf97941d77,345,0,8,67,1613,3356,0,0,True,1,1966.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106825,822a1ca844fc0be5e911aaa8d5196928,b863478e836bf746348b4c92ac744d30,361,1,52,87,521,1856,0,0,False,2,1985.0,0,0
1106826,822a1ca844fc0be5e911aaa8d5196928,f80511b6f9315b26b001cf523feed49d,315,0,10,289,1795,2451,0,0,False,2,1985.0,0,0
1106833,30dfbb64c2809ca9b1167a6a7805c713,9708eee62a5d3395d9ea67f4ec98f91a,359,1,23,338,987,3911,1,0,False,2,NaN,0,0
1106834,30dfbb64c2809ca9b1167a6a7805c713,9963a36b48c83b6c859692cd8603599f,359,0,10,289,990,34,0,0,False,2,NaN,0,0


In [165]:
transactions_sampled

,Unnamed: 0,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,30,9645891030b4a3d0dee526c47dcbcd13,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,11,4d68fdddb5deaa496ce485bd51929ab1,d66c44ca2800163901f22ec5fb4f1021,0.871,261.29,1
1,200,cc23c42be0936fb84cdac81ecb5d4ac2,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,14,260e10d8bcda8c88978d9809d7538261,f2431e06709a50ee05efc2e9ef3f41e7,1.000,44.99,0
2,209,478598ef73bb6f2f2d8e3980c92d9193,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,1,439d7c61c4016ddd027a204853dfe65c,1284bfe52154a7510bb440f9ce2e54aa,1.000,329.99,0
3,219,58b442bb93c8823fa14c3a7406c71a23,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,27,3ec3be094051d958c755e0f5154c678c,c8fa3becd13b8d45156ac55ab82d93a5,0.306,149.94,0
4,226,58b442bb93c8823fa14c3a7406c71a23,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,2,3ec3be094051d958c755e0f5154c678c,f93383168e8c685f3e0d8ebb3c4ab16e,1.000,239.99,0
...,...,...,...,...,...,...,...,...,...,...
1110575,32109249,71530873e67284bd64cd4988ef3cf396,8257a28c7042c73d88d2404f4806cbf4,2016-10-31,1,25a9754c8ab90670ff8852d124939ed1,3afa2c41341548499934a52d7338f33e,1.000,6.79,0
1110576,32109259,dcc09c1480fd3e0419b474d0152ed696,8257a28c7042c73d88d2404f4806cbf4,2016-10-31,1,c9487b0580dea0b1418ae6413f2fb479,f0be85f7705ffbd2e4b2cb97a054b56a,1.000,54.00,1
1110577,32109318,b7d414fa2ea0715c81042edca1deaab1,46ce6c3c1e23ab2ed612a1a43ebb2929,2016-10-31,12,bf53879cd71225fc85cb8268196f590f,6278f48a43fb0a23036b3a11c92e726c,0.668,42.68,1
1110578,32109379,5f77812e75307c05b665911839c23693,a300549b1292442df6593433d411cead,2016-10-31,3,8bff61bf078e1836cee7122ac1ae4988,cad2b7286d60ec58918d088503b518df,0.152,7.58,0


In [175]:
merged_predicted_inverse = transactions_sampled.merge(reversed_test, on=['client_id', 'material'])

In [207]:
merged_predicted_inverse.label

0         1
1         1
2         1
3         1
4         1
         ..
212029    1
212030    1
212031    1
212032    0
212033    0
Name: label, Length: 212034, dtype: int64

In [174]:
instock_frame = pd.read_csv('instock.csv').drop(columns=['Unnamed: 0'])
instock_dict = defaultdict(set)

for _, row in tqdm.tqdm(instock_frame.iterrows()):
    for i, val in enumerate(row[2:]):
        if val > 0:
            instock_dict[row[0] + row[1]].add(row.index[i + 2])

23596it [01:08, 342.97it/s]


In [181]:
def check_in_stock(row):
    mat_id = row.plant + row.material
    if instock_dict.get(mat_id) is None:
        return True
    return row.chq_date_x in instock_dict[mat_id]

In [182]:
merged_predicted_inverse.apply(check_in_stock, axis=1)

0          True
1          True
2         False
3         False
4          True
          ...  
212029    False
212030     True
212031    False
212032    False
212033    False
Length: 212034, dtype: bool

In [199]:
merged_predicted_inverse[merged_predicted_inverse.chq_date_x == '2017-08-25']

,Unnamed: 0,chq_id,plant,chq_date_x,chq_position,client_id,material,sales_count,sales_sum,is_promo,...,hier_level_3,hier_level_4,vendor,is_private_label,is_alco,gender,city,birthyear,label,importance
1,8518661,65c4aa82f8c72d825bc86a1882f583f3,0f3de8bf2672002182482b5a55464b13,2017-08-25,1,bbfb43daf80f1e5caad1f31b5d550984,b75147feae4cf8ec07fb5f2cef855def,1.0,3.79,0,...,303,688,1381,1,0,False,1,1987.0,1,1
97,28256127,8c7923b99d4a67d8dbf7e2344fd30ac5,5359c847ef4975cdc646e8be8cb5621f,2017-08-25,8,72ad70c18abcb8151d3098338390d029,564840544bef2c259cf7a0642ab97b2e,1.0,53.99,0,...,15,847,1378,1,0,False,1,1948.0,1,1
98,28256127,8c7923b99d4a67d8dbf7e2344fd30ac5,5359c847ef4975cdc646e8be8cb5621f,2017-08-25,8,72ad70c18abcb8151d3098338390d029,564840544bef2c259cf7a0642ab97b2e,1.0,53.99,0,...,15,847,1378,1,0,False,1,1948.0,1,1
177,28245324,1197588722042a4c005d141804f9c7ab,84f7b02a8128f5f5775611244c24b941,2017-08-25,1,e147e71462765827362bbe8437f16107,b75147feae4cf8ec07fb5f2cef855def,3.0,10.47,0,...,303,688,1381,1,0,False,2,1981.0,1,1
373,28237052,2fcc32e86757a9fe8dce8731072cb60e,85d606c4530d98879e87f1be4aa56fc2,2017-08-25,2,43a8fab4b0bc01d3fc50485fa3ad0183,58f3735e298a6cff28fba744e8a5f322,4.0,288.04,0,...,60,1922,3845,1,0,True,1,1958.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150462,28257208,c83779055174f109c4dfa0ed233cb417,238d77348d954fdd455aabb09575f2d0,2017-08-25,8,da7fbf7514e484da3dcc46f50fce81f2,72ddaea10db500a90cb3e89376229e6e,1.0,27.19,0,...,267,966,207,0,0,False,2,1976.0,1,1
150463,28257227,de8d2cbf7796dce4d25a55be85eea9ed,6e1acaeb838a890e6570284f73e1d00f,2017-08-25,2,5b55d66c80c246a1daf8b12b3d438645,8b7b62bbea5b8975087458ddae8d041b,1.0,106.00,0,...,172,598,1524,0,0,False,2,1990.0,1,1
150464,28257272,248c7612220f427552be48df65ac65a1,6e1acaeb838a890e6570284f73e1d00f,2017-08-25,14,0c96bb6e56c357962466d11875c94469,53a5a6521ce6021b2259cd2ffd1b9dc7,1.0,48.86,1,...,183,360,969,0,0,False,2,1958.0,1,1
150465,28257277,1b743cd07f98d06cc5055fff9e61c495,6e1acaeb838a890e6570284f73e1d00f,2017-08-25,4,ed3673d58d2d47c5d0269a43aa590d1c,6166da0d2cfb0bbdd7b621e4b3e95f7e,1.0,27.77,0,...,213,726,3952,0,0,False,2,1959.0,1,1


In [203]:
instock_frame[['plant', 'material', '2017-08-25']]

,plant,material,2017-08-25
0,00fdea78e677259a3e44868468f24bbf,0b689fd70e79d5010e397b4c453e8460,0
1,00fdea78e677259a3e44868468f24bbf,0de2d4c7997862a3cb3fa23409a0171e,0
2,00fdea78e677259a3e44868468f24bbf,0e2823baca6c23c591c96e7399a74ae6,0
3,00fdea78e677259a3e44868468f24bbf,0e31ad075d99fab63be09dad89580d71,0
4,00fdea78e677259a3e44868468f24bbf,119a9ea3a832472e1c163e5c1afba59e,0
...,...,...,...
23591,ff1e1ec06b812e935f5875c40924d7fc,6278f48a43fb0a23036b3a11c92e726c,1
23592,ff1e1ec06b812e935f5875c40924d7fc,b75147feae4cf8ec07fb5f2cef855def,1
23593,ffe49cdf45d56b04b8542e6abc4f5fb6,3afa2c41341548499934a52d7338f33e,1
23594,ffe49cdf45d56b04b8542e6abc4f5fb6,6278f48a43fb0a23036b3a11c92e726c,0


In [185]:
merged_predicted_inverse

,Unnamed: 0,chq_id,plant,chq_date_x,chq_position,client_id,material,sales_count,sales_sum,is_promo,...,hier_level_3,hier_level_4,vendor,is_private_label,is_alco,gender,city,birthyear,label,importance
0,1039,f4867385149b52425628d3d2c8f92697,6950aac2d7932e1f1a4c3cf6ada1316e,2016-11-01,1,bbfb43daf80f1e5caad1f31b5d550984,b75147feae4cf8ec07fb5f2cef855def,1.000,3.49,0,...,303,688,1381,1,0,False,1,1987.0,1,1
1,8518661,65c4aa82f8c72d825bc86a1882f583f3,0f3de8bf2672002182482b5a55464b13,2017-08-25,1,bbfb43daf80f1e5caad1f31b5d550984,b75147feae4cf8ec07fb5f2cef855def,1.000,3.79,0,...,303,688,1381,1,0,False,1,1987.0,1,1
2,4097,121538c8a797732395e655c7f203b9ff,2aceac02230cdeb39aaf319ac6779462,2016-11-01,1,1d64234079fc2d5635b00aa801cf6629,2f588cc938971a6efdfe762392e31d3c,2.000,257.62,0,...,61,710,1819,0,0,False,2,1963.0,1,0
3,27761932,747da843b92609b0c97d2fd938dee3d2,2aceac02230cdeb39aaf319ac6779462,2017-08-17,17,1d64234079fc2d5635b00aa801cf6629,2f588cc938971a6efdfe762392e31d3c,2.000,197.98,1,...,61,710,1819,0,0,False,2,1963.0,1,0
4,13088,7f015417f15f865bba648cbcb105fd04,54e0f6e4ee558d9346d379b4ca2a25a0,2016-11-01,1,faa471316a5b6976273c59decfd26c76,3afa2c41341548499934a52d7338f33e,2.000,13.58,0,...,303,688,1381,1,0,False,2,1991.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212029,30228647,98087f17234f87a16d5e84ae7e9b226d,8a5dd78aadea6ffc984bda22064adb92,2017-10-03,7,9da09afdb6607191389d5f30274710cd,e6b6bb5e0332799839019f6e132c1017,1.000,98.89,0,...,170,614,3077,0,0,False,2,1952.0,1,1
212030,30228653,98087f17234f87a16d5e84ae7e9b226d,8a5dd78aadea6ffc984bda22064adb92,2017-10-03,12,9da09afdb6607191389d5f30274710cd,6278f48a43fb0a23036b3a11c92e726c,1.124,63.94,0,...,376,1171,1386,0,0,False,2,1952.0,1,1
212031,30228669,9a6660b2b3797d87bdb1138c9c87b684,8a5dd78aadea6ffc984bda22064adb92,2017-10-03,4,9810a8dcc0d728e60af3e86177fae709,d5e56b631814d9553852715bc5ceb8e3,1.000,24.99,1,...,83,1185,1221,0,0,False,2,1948.0,1,1
212032,31352112,9f61adb2a56ba76525c6d5a86dd62420,45226bae7da31547fbdca1c62d24a8dd,2016-10-20,4,b09b1c7e1eeb6379e102ede0e606ee3b,d688ebcbd7c01ae0eaf214c9a82c1b88,1.000,24.19,0,...,350,1804,930,0,0,False,2,NaN,0,1


In [ ]:
instock_frame[(instock_frame.plant==plant) | (instock_frame[day] == 0)].material.unique()

In [330]:
day = '2017-09-01'

plant = '2aceac02230cdeb39aaf319ac6779462'


clients = merged_predicted_inverse[(merged_predicted_inverse['plant'] == plant) & (merged_predicted_inverse['chq_date_x'] != day)].client_id.unique()

absent_items = instock_frame[(instock_frame.plant==plant) | (instock_frame[day] == 0)].material.unique()

points = []
for item in absent_items:
    for client in clients:
        points.append((client, item, day))
        

In [331]:
len(clients), len(absent_items)

(199, 2118)

In [332]:
negative_frame = pd.DataFrame(points, columns=['client_id', 'material', 'chq_date'])

In [333]:
trial_points = negative_frame.merge(materials_enc, on='material').merge(client_frame, on='client_id')
trail_points = fix_date(trial_points)
trail_points['client_id'] = clients_enc.transform(trail_points.client_id)
trail_points['material'] = mat_enc.transform(trail_points.material)

In [334]:
trail_points

,client_id,material,chq_date,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco,gender,city,birthyear
0,10536,3416,332,0,13,81,3,3534,0,1,False,2,1963.0
1,10536,3472,332,0,49,313,125,2674,0,0,False,2,1963.0
2,10536,4325,332,0,26,93,1894,3825,0,0,False,2,1963.0
3,10536,4453,332,0,13,81,3,2715,0,1,False,2,1963.0
4,10536,6723,332,0,35,366,1861,1999,0,0,False,2,1963.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
421278,87455,40842,332,0,21,15,847,3119,0,0,False,2,1956.0
421279,87455,42976,332,0,28,267,966,2594,0,0,False,2,1956.0
421280,87455,43109,332,0,32,219,1332,922,0,0,False,2,1956.0
421281,87455,43953,332,0,28,267,1668,436,0,0,False,2,1956.0


In [335]:
X = trial_points.to_numpy()

In [336]:
y_pred_absent = boost.predict(X)

In [337]:
trail_points['importance'] = y_pred_absent

In [340]:
wanted = trail_points[trail_points.importance == 1]

In [354]:
wanted_counts = wanted.groupby('client_id').count()['material']
wanted_counts[wanted_counts > 1500].index

Int64Index([  505,  1495,  2251,  2630,  6439,  6703, 15175, 16646, 19442,
            24429, 24879, 30000, 32258, 34897, 35455, 35764, 37433, 39238,
            41152, 43598, 46835, 47671, 50869, 53018, 58566, 63377, 71159,
            73639, 73950, 75273, 77192, 81848, 84567, 87570, 88185, 90505,
            90564, 90976, 91966],
           dtype='int64', name='client_id')

In [360]:
missed_guys = clients_enc.inverse_transform(wanted_counts[wanted_counts > 1500].index)

In [5]:
missed_guys = np.array(['01742ebdb1ceea185feceadcd8adcee7',
       '0438348c515c19d933831bec1cc1d472',
       '0643672bc38f396c63eade84873089d9',
       '0747aaa37d462a90b4facb6212776108',
       '1214110e2a682607c15ee0ff419514d7',
       '12c024a0da33be50f48d90b94249d6fa',
       '2a2554ac431c95d6b98823322bac9ab1',
       '2e3eb384999442dd199fdf3a4c0d9463',
       '364173053bc689ce9f066ff3b3c63619',
       '443fc94f1f1c1ecabfe50a3bdce2bfc7',
       '458782935990b03ec7ddf27ffc7ad95a',
       '53cf6ba19ddbe0609da034b13b0e8030',
       '5a2c907344800f6573df7229b6847cf3',
       '6175f0aeb5b3a5b2054b16f7e2d6db1e',
       '62eb27f5ebba590131416c20c266ef8a',
       '63d4c707a9d5a6264c0f19f83b057f6d',
       '6843d220891dfb09767a8d7e719288d5',
       '6d565aab852aa329dcf5c1eb9d46d8eb',
       '72c3b4ac505b2087953126994006a3d1',
       '79927bfe23610509d2a4731a28c1c966',
       '8297e06b9d0762ffeaebe4f750a8471f',
       '84f33d48a91b8b82051233d566eb61f9',
       '8dd1569ab4ccefb62edb9ad92c1cd352',
       '93ac385fb056e2d18255885d6fc2cd27',
       'a325303f74c6699dac446c786ab8d256',
       'b070c97c5cff17965e958781fec5664b',
       'c61f8b89cbd5110d863739e1ea20e63e',
       'ccc52dc26bc23320079a63017f7d9b6c',
       'cd8e7c9d39a8b0871834f9724578547d',
       'd136a031b07f474a877be41f353dc579',
       'd6612b8b11866d713b4adb2570e8bb2b',
       'e30acbe28af7cd241123976e55f8cea2',
       'eab24fef6bb1aca7059f5531915f64a6',
       'f2f1e1acef187e4e021cfdfb9c698a85',
       'f4aace0295afc7a5d9ded9420905c439',
       'fb1d5aa2e4fe656fc8dcb2f5d5e47f1f',
       'fb4c2f7aaf76fa6fb113ad8cf0365581',
       'fc772273845707de231d69e5e00048ad',
       'ff30a1b016016cb2c89c5826dcb56f89'])

In [11]:
mean_loss = 0.0
for guy in tqdm.tqdm(missed_guys):
    mean_loss += transactions_data[transactions_data.client_id == guy].groupby('chq_id').sum().mean()['sales_sum']

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [01:11<00:00,  1.83s/it]


In [12]:
mean_loss

55593.04334909091

In [13]:
monthly = mean_loss * 30
monthly

1667791.3004727273

In [30]:
transactions_data[transactions_data.plant == '6bcf53c2bdbee891ffaca1e612b0076c'].sales_sum.sum() / 12

2443294.27

In [26]:
transactions_data.plant.unique()[:20]

array(['7cd86ecb09aa48c6e620b340f6a74592',
       '6950aac2d7932e1f1a4c3cf6ada1316e',
       'b6fb522815d06fed82b0140be4c74680',
       '857778a20b9a41d4ca0d687a36e4bfa8',
       'd9f5e405a7f74ed652a8f0b31a87f636',
       'ed0a75eeb69b34ddc14beed2678bee12',
       '0e0b24fc303d2b384be5a2464654a5d2',
       'c90070ff096dd6858022784617b2f383',
       '8ae502a489e46e16512581742d93db9c',
       '99cb58d2d19256c97a9055c0675bcb27',
       '0bb0666df4640d65ed353b3bfca34d6e',
       '2d0974b2436345d99f7152c3fc44a609',
       '028f7ee9eb0c0994a8ba1e60bcffd09e',
       'b2c3b75b24f611bde67921d19c8dffcd',
       '7792dd2aad4d5547e11d9317b03c32c6',
       '2aceac02230cdeb39aaf319ac6779462',
       'b323b8172b421577bf8e70d411d79a50',
       'c76b3d3ab1522b66814253d642932e9e',
       '021ed42efda232c8b396e590360a5b89',
       '6bcf53c2bdbee891ffaca1e612b0076c'], dtype=object)